testing on my Windows laptop

In [1]:
import matplotlib.pyplot as plt
from databroker import Broker
import snapshot

from bluesky.utils import install_nb_kicker  # for notebooks, qt -> nb
install_nb_kicker()
plt.ion()

from bluesky import RunEngine
RE = RunEngine({})

mongo_host = "poof"
cfg = {
    'metadatastore' : {
        'module' : 'databroker.headersource.mongo',
        'class' : 'MDS',
        'config' : {
            'host': mongo_host,
             'port': 27017,
             'database': 'metadatastore-production-v1',
             'timezone': 'US/Central'},
    },
    'assets' : {
        'module' : 'databroker.assets.mongo',
        'class' : 'Registry',
        'config': {
            'host': mongo_host, 
            'port': 27017, 
            'database': 'filestore-production-v1'}
    },
}
db = Broker.from_config(cfg)
RE.subscribe(db.insert)

0

In [2]:
#h = db[-1]
#h.start

In [3]:
from bluesky import plans as bp
from bluesky import callbacks as bc
from ophyd import Signal, EpicsSignal

reporter = snapshot.SnapshotReport()
signal = Signal(name="signal")

In [4]:
RE(bp.count([signal]), bc.LiveTable([signal]))
h = db[-1]
h.table()

+-----------+------------+------------+
|   seq_num |       time |     signal |
+-----------+------------+------------+
|         1 | 23:55:56.8 |      0.000 |
+-----------+------------+------------+
generator count ['4b0ab337'] (scan num: 1)


,time,signal
seq_num,,
1,2018-12-19 23:55:56.876663923,0.0


In [5]:
RE(snapshot.snapshot([signal], md=dict(purpose="non-EPICS data")), bc.LiveTable([signal]))
h = db[-1]
reporter.print_report(h)
h.table()

+-----------+------------+------------+
|   seq_num |       time |     signal |
+-----------+------------+------------+
|         1 | 23:55:57.2 |      0.000 |
+-----------+------------+------------+
generator snapshot ['3a71a356'] (scan num: 2)
snapshot: 2018-12-19 23:55:57.233482

hints: {}
iso8601: 2018-12-19 23:55:57.233482
plan_description: archive snapshot of ophyd Signals (usually EPICS PVs)
plan_name: snapshot
plan_type: generator
purpose: non-EPICS data
scan_id: 2
time: 1545285357.2344792
uid: 3a71a356-0361-43b3-bebe-fdd0eb97a307

====== ====== ===== ==========================
source name   value timestamp                 
====== ====== ===== ==========================
SIM    signal 0.0   2018-12-19 23:55:56.712549
====== ====== ===== ==========================




,time,signal
seq_num,,
1,2018-12-19 23:55:57.264399767,0.0


In [6]:
pvlist = "xxx:SR_heartbeat  xxx:HEARTBEAT".split()
obj_dict = snapshot.connect_pvlist(pvlist)
RE(snapshot.snapshot(obj_dict.values(), md=dict(purpose="EPICS data")), bc.LiveTable(obj_dict.values()))
h = db[-1]
reporter.print_report(h)
h.table()

+-----------+------------+------------+------------+
|   seq_num |       time |   signal_0 |   signal_1 |
+-----------+------------+------------+------------+
|         1 | 23:55:57.7 |          0 |       4511 |
+-----------+------------+------------+------------+
generator snapshot ['31dbb5ef'] (scan num: 3)
snapshot: 2018-12-19 23:55:57.592694

hints: {}
iso8601: 2018-12-19 23:55:57.592694
plan_description: archive snapshot of ophyd Signals (usually EPICS PVs)
plan_name: snapshot
plan_type: generator
purpose: EPICS data
scan_id: 3
time: 1545285357.5926945
uid: 31dbb5ef-1a87-42f4-accb-f13b2ffc4609

====== ================ ====== ==========================
source name             value  timestamp                 
====== ================ ====== ==========================
PV     xxx:HEARTBEAT    4511.0 2018-12-19 23:55:57.317774
PV     xxx:SR_heartbeat 0      2018-12-19 23:55:57.561990
====== ================ ====== ==========================




,time,signal_1,signal_0
seq_num,,,
1,2018-12-19 23:55:57.738303423,4511.0,0
